In [0]:
import numpy as np
#%matplotlib inline
#import matplotlib.pyplot as plt
from PIL import Image  , ImageDraw 
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import os
import json

In [0]:
torch.__version__

'1.5.0+cu101'

In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd drive/My Drive/Resnet_Beta-Version
!ls

/content/drive/My Drive/Resnet_Beta-Version
 acc
'alpha-Resnet(Raw+Face+Hand).ipynb'
'alpha-Resnet(Raw+Face).ipynb'
'alpha-Resnet(Raw+Hand).ipynb'
'alpha-Resnet(Raw).ipynb'
'Arrange Dataset.ipynb'
'beta-Resnet(Raw+Face+Hand).ipynb'
'beta-Resnet(Raw+Face+Hand+Pose)-Pose as Depth.ipynb'
'beta-Resnet(Raw+Face).ipynb'
'beta-Resnet(Raw+Hand).ipynb'
'beta-Resnet(Raw).ipynb'
 dataset
 dump
 epoch_65.pth
 extract_faceandhand_jorehgaye.ipynb
 fallingDemo.ipynb
 fallingDemo_modified_2.py
 fallingDemo_modified.py
 fallingDemo.py
'gamma-Resnet(Raw+Face+Hand).ipynb'
'gamma-Resnet(Raw+Face+Hand+Pose)-Pose as Depth.ipynb'
'gamma-Resnet(Raw+Face).ipynb'
'gamma-Resnet(Raw+Hand).ipynb'
'gamma-Resnet(Raw).ipynb'
 loss
 out.jpg
 putTXT_rightplace.ipynb
'Testbeta-Resnet(Raw+Face+Hand).ipynb'
'Testbeta-Resnet(Raw+Face+Hand+Pose)-Pose as Depth.ipynb'
'Testbeta-Resnet(Raw+Face).ipynb'
'Testbeta-Resnet(Raw+Hand).ipynb'
'Testbeta-Resnet(Raw).ipynb'
 TestingModels_first_gamma.ipynb
 TestingModels_first.ipynb
 Te

In [0]:

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
}

In [0]:

class MyImageFolder(datasets.ImageFolder):
    
    def __init__(self, root, transform=None):
        super(MyImageFolder,self).__init__(root)

        self.transform = transform

    def __getitem__(self, index):

        #imgs = self.imgs[index]
        #print(imgs)

        path, target = self.samples[index]
        #sample = self.loader(path)
        #print("Sample: ",sample)
        

        hm_transform = transforms.Compose([
                          transforms.Resize((224,224)),
                          transforms.ToTensor()

                                ])
        self.transform = hm_transform
        #print(path)
        #print(type(path))
        #all_images = []
        #all_labels = []
        #all_hm_face = []
        #all_hm_hand = []
        # target_acquired - 0: image path -- 1: label
        #for i in len(path):

        img_path = path
        #print("--i: %d , TA: %s"%(i,target_acquired))

        
        #img_path = target_acquired
        #label =  target_acquired[1]

        #print(img_path)

        image = Image.open(img_path)
        img_w , img_h = image.size

    
        filename = os.path.basename(img_path)
        ann_path = img_path.replace(filename, filename.split('.')[0])
        hm_face , hm_hand = self.get_heatmap(ann_path + '.txt',img_w,img_h)  
        hm_pose = self.get_heatmap_pose(ann_path + '.json',img_w,img_h)
        #image = np.dstack((image, hm_face))
        cpath  = img_path.replace(filename, filename.split('.')[0] + '-bbox.txt')
         
        image,hm_face,hm_hand,hm_pose = self.crop_text(cpath,image,hm_face,hm_hand,hm_pose)
        # Transforms
        if self.transform is not None:
            
            image = self.transform(image)
            hm_face = hm_transform(hm_face)
            hm_hand = hm_transform(hm_hand)
            hm_pose = torch.stack([ hm_transform(x) for x in hm_pose ], dim=1).squeeze(0)


        #print( hm_pose.size() )
        image = torch.cat((image,hm_face,hm_hand,hm_pose))
       
        #print(image.size())
        #if self.target_transform is not None:
        #    target = self.target_transform(target)
        
        
        # For Face
            #image = np.dstack((image, hm_face))

            #all_images.append(img_path)
            #all_labels.append(label)
            #all_hm_face.append(hm_face)
            #all_hm_hand.append(hm_hand)

        #return path, all_hm_face, all_hm_hand, target
        
        return image, target

    def crop_text(self,path,image,hm_face,hm_hand,hm_pose):
        try:
              fr = open(path)
              # print ("fr",cat+"/yolo/" + imagePath.split("/")[-1].split('.')[0]+".txt")

              line = fr.readline().split(" ")

              width, height = image.size 
        
              # Setting the points ffloor cropped image
              b_w = float(line[3]) * width
              b_h = float(line[4]) * height

              t_left = (float(line[1]) * width) - (b_w/2)
              t_right = (float(line[2]) * height) - (b_h/2)

              b_left = (float(line[1]) * width) + (b_w/2)
              b_right = (float(line[2]) * height) + (b_h/2)

              # print("w",width," h",height)

              # print (t_left)
              # print (t_right)
              # print (b_left)
              # print (b_right)

              image = image.crop((t_left, t_right, b_left, b_right))
              
              hm_face = hm_face.crop((t_left, t_right, b_left, b_right))
              hm_hand = hm_hand.crop((t_left, t_right, b_left, b_right))
              
              for i in range(len(hm_pose)):
                  hm_pose[i] = hm_pose[i].crop((t_left, t_right, b_left, b_right))

        except FileNotFoundError:
              print("FileNotFoundError: " + path)
              #pass
        return image ,hm_face,hm_hand,hm_pose

    def get_heatmap(self,ann_path, img_w , img_h):
        
        hm_0 = Image.new('L', (img_w, img_h))
        hm_1 = Image.new('L', (img_w, img_h))
        
        try:

            with open(ann_path) as f:

                for line in f.readlines():

                    if line == '\n':
                        continue

                    line = line.split(' ')

                    cx = float(line[1])
                    cy = float(line[2])
                    w = float(line[3])
                    h = float(line[4])

                    xmin =  (cx - (w/2)) * img_w
                    ymin = (cy - (h/2)) * img_h
                    xmax = (cx + (w/2)) * img_w
                    ymax = (cy + (h/2)) * img_h

                    if line[0] == '0':
                        hmDraw = ImageDraw.Draw(hm_0)   
                    else:
                        hmDraw = ImageDraw.Draw(hm_1)   
                    
                    hmDraw.rectangle([xmin,ymin,xmax,ymax], fill ="#ffffff")
        except FileNotFoundError:
            #print("FileNotFoundError: %s"%ann_path)
            pass

        return hm_0, hm_1

    def get_heatmap_pose(self,ann_path, img_w, img_h):
        
        heatmaps = [ Image.new('L', (img_w, img_h)) for x in range(17) ]

        try:
            with open(ann_path,'r') as f:
                data = json.load(f)
            #print(ann_path)
            #print(data)  
            try:
                keypoints = data[0]['keypoints']
            except IndexError:
                return heatmaps

            #print(keypoints)
            # TODO check for two key mappings -- working

            for i in range(0,len(keypoints),3):

                xmin = keypoints[i] - 5
                ymin = keypoints[i+1] - 5
                xmax = keypoints[i] + 5
                ymax = keypoints[i+1] + 5

                hmDraw = ImageDraw.Draw(heatmaps[i//3]) 
                hmDraw.rectangle([xmin,ymin,xmax,ymax], fill ="#ffffff")

        except FileNotFoundError:
            #print("FileNotFoundError: %s"%ann_path)
            pass

        return heatmaps



"""
Test Code


path, target = self.samples[index]
sample = self.loader(path)
if self.transform is not None:
  sample = self.transform(sample)
if self.target_transform is not None:
  target = self.target_transform(target)
# this is where you load your resnet data
resnet_path = os.path.join(os.path.splitext(path)[0], '.pth')  # replace image extension with .pth
resnet = torch.load(resnet_path)  # load the stored features
return sample, resnet, target

#dataset = MyImageFolder("falling_dataset/val/")
#dataset[0]
#h = dataset.get_heatmap_pose("falling_dataset/train/Falling/11107.json",1280,720)
#print(len(h))
#for i in range(len(h)):
#    h[i].save("Annotated/train/%d.jpg"%(i))

path = "./falling_dataset/train/Falling"
!pwd
for file in os.listdir(path):
    if file.endswith(".json"):
        #print(os.path.join(path, file))

        with open(os.path.join(path, file)) as f:
            string = f.readline()
            if string == '[]':
                print(os.path.join(path, file))



#re, labels = dataset[1:3]
#print(re)
#print(labels)
#re[2].save('pred_face.jpeg')
#re[3].save('pred_hand.jpeg')
"""

'\nTest Code\n\n\npath, target = self.samples[index]\nsample = self.loader(path)\nif self.transform is not None:\n  sample = self.transform(sample)\nif self.target_transform is not None:\n  target = self.target_transform(target)\n# this is where you load your resnet data\nresnet_path = os.path.join(os.path.splitext(path)[0], \'.pth\')  # replace image extension with .pth\nresnet = torch.load(resnet_path)  # load the stored features\nreturn sample, resnet, target\n\n#dataset = MyImageFolder("falling_dataset/val/")\n#dataset[0]\n#h = dataset.get_heatmap_pose("falling_dataset/train/Falling/11107.json",1280,720)\n#print(len(h))\n#for i in range(len(h)):\n#    h[i].save("Annotated/train/%d.jpg"%(i))\n\npath = "./falling_dataset/train/Falling"\n!pwd\nfor file in os.listdir(path):\n    if file.endswith(".json"):\n        #print(os.path.join(path, file))\n\n        with open(os.path.join(path, file)) as f:\n            string = f.readline()\n            if string == \'[]\':\n                pr

In [0]:
input_path = './dataset/alpha/Fall'

image_datasets = {
    'train': 
    MyImageFolder(input_path + '/train', data_transforms['train']),
    'validation': 
    MyImageFolder(input_path + '/val', data_transforms['validation'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=64,
                                shuffle=True,
                                num_workers=2),  # for Kaggle
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=32,
                                shuffle=False,
                                num_workers=2)  # for Kaggle
}

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [0]:
model = models.resnet50(pretrained=True).to(device)

model.cuda()
#print(model)

depth = 22    # ( 3 + 1 + 1 + 17 )


for param in model.parameters():
    param.requires_grad = False   


#------------ Modify First Layer
model.conv1 = nn.Conv2d(depth, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False).to(device)
model.conv1.requires_grad = True

#------------ Modify Last Layer
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2),
               nn.Sigmoid()).to(device)
#epoch = 87
#model = (torch.load('weights/gamma/Fall/raw-face-hand-pose/epoch_%d.pth'%epoch))

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(),lr=0.001)

In [0]:
def train_model(model, criterion, optimizer, num_epochs=3):
    new_write = True
    for epoch in range(num_epochs):
        
        f = open('loss/gamma/Choke/raw-face-hand-pose.txt','a+')
        
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        f.write('Epoch {}/{}\n'.format(epoch+1, num_epochs))
        

        for phase in ['train','validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            count = 0
            
            with torch.set_grad_enabled(phase == 'train'):
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    if phase == 'train':

                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                    _, preds = torch.max(outputs, 1)
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                    count = count+1                
                    if (count%40==0):
                        print (count)
             

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
            if (os.path.isfile('loss/gamma/Choke/raw_face_hand_pose.txt')):
              new_write=False  
            else :
              print ("No file")
            f = open('loss/gamma/Choke/raw_face_hand_pose.txt','a+')
            if  (new_write == True ):
              print ("here")
              f.write('{} loss: {:.4f}, acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
            else :
              f.write ('\n')
              f.write('{} loss: {:.4f}, acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
            
            
        torch.save(model, '/content/drive/My Drive/Fyp/gamma/Choke/raw-face-hand-pose/epoch_%d.pth'%(epoch))
        f.close()


    return model

In [0]:

train_model(model, criterion, optimizer, num_epochs=100)

Epoch 1/100
----------
FileNotFoundError: ./dataset/alpha/Choke/train/NonChoking/VID_20200415_171633(0)_frame-910-bbox.txt
FileNotFoundError: ./dataset/alpha/Choke/train/NonChoking/VID_20200415_165905(0)_frame-30-bbox.txt
40
FileNotFoundError: ./dataset/alpha/Choke/train/NonChoking/VID_20200415_171633(0)_frame-915-bbox.txt
FileNotFoundError: ./dataset/alpha/Choke/train/NonChoking/VID_20200415_165905(0)_frame-35-bbox.txt
FileNotFoundError: ./dataset/alpha/Choke/train/NonChoking/VID_20200415_165905(0)_frame-40-bbox.txt
FileNotFoundError: ./dataset/alpha/Choke/train/NonChoking/VID20200419005215_220-bbox.txt
train loss: 0.6419, acc: 0.6410
No file
here
FileNotFoundError: ./dataset/alpha/Choke/val/NonChoking/VID_20200415_171633(0)_frame-925-bbox.txt
validation loss: 0.6493, acc: 0.5857
Epoch 2/100
----------
FileNotFoundError: ./dataset/alpha/Choke/train/NonChoking/VID_20200415_165905(0)_frame-35-bbox.txt
FileNotFoundError: ./dataset/alpha/Choke/train/NonChoking/VID20200419005215_220-bbox.t